# 字母级语言模型 - 恐龙领地

简单地说，就是使用一个模型来给恐龙起名字。
你的助理已经收集了关于恐龙名字的一个列表，并且放进一个数据库里面。为了创造新的名字，你将建立一个字母级别的语言模型，你的算法讲学习不同的名字模式，并且随机生成新的名字。

通过这一节任务将学习到：
1. 如何存储文本数据来使用RNN模型处理
2. 如何合成数据，通过对每一个时间步的预测结果进行取样，并且将结果传递到下一个RNN单元
3. 如何创建一个字母级别的文本生成RNN网络
4. 为什么裁剪梯度很重要

In [4]:
import numpy as np
from utils import *
import random
from random import shuffle

## 1. 问题陈述

### 1.1 数据集和预处理
下面的代码可以读取恐龙名字的数据集，创建一个唯一的字母列表，比如a-z，并且计算数据集和单词长度

In [5]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


27个字符包括了a到z 26个字母加上“\n”，它的作用就相当于`<EOS>`,表示一个名字的结束。接下来创建一个字典，将这27个字符映射为0-26的数字，以及另一个字典从数字到字母的映射。

In [6]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 1.2 模型结构构思

模型将有以下结构：
1. 初始化参数
2. 运行优化循环
    - 正向传播计算损失函数
    - 反向传播计算梯度值
    - 修剪梯度值避免梯度爆炸
    - 使用梯度值，来更新参数根据梯度下降法的更新规则
3. 返回学习到的参数

<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center> **Figure 1**: 循环神经网络</center></caption>

在每一个时间步，都尝试在给定上一个输出字母的情况下下一个单元的输出字母是什么。数据集$X=(x^{<1>},x^{<2>},....,x^{<T_x>})$是训练集中的一个字母列表，同时输出$Y=(y^{<1>},y^{<2>},....,y^{<T_y>})$也是，并且有$x^{<T+1>} = y^{<T>}$ 

## 2 创建模型的基本块

这一部分，将建立整个模型用到的两种重要块：
- 梯度修剪： 避免梯度爆炸
- 取样：生成字母的一种技巧

### 2.1 修剪梯度 in the optimization loop

实现`clip`函数，你的整体循环结构里面通常都包括，正向传播，计算代价，反向传播和更新参数的过程。在更新参数前，就需要执行梯度修剪，来保证梯度不会爆炸。

这个`clip`函数需要输入一个所有梯度值的字典，然后返回一个修剪过后的版本。 有很多的修剪梯度的方法，将使用简单的元素对应的修剪小程序，梯度向量的每一个UAN素都会被修剪到在范围$[-N,N]$之间。通常有个最大值，加入超过这个最大值，那么就设置为正值。如果小于-N，那么就设置为-N。

<img src="images/clip.png" style="width:400;height:150px;">
<caption><center> **Figure 2**: 有无修剪过程的可视化效果</center></caption>


In [7]:
### GRADED FUNCTION: clip

def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    ### START CODE HERE ###
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient,-maxValue , maxValue, out=gradient)
        # 使用numpy里面自带的函数 out = gradient 保证数组中的内容直接替换
    ### END CODE HERE ###
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [8]:
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]


### 2.2 采样

假设模型已经训练过了，现在用来生成新的字母，过程如下图：
<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center> **Figure 3**: 假设模型训练过了，传递$x^{<1>} = \vec{0}$ 在第一步时，然后采样出一个字母 </center></caption>

采样实现的4个步骤：
1. 设置$x^{<1>} = \vec{0}$和$a^{<0>} = \vec{0}$，默认输入
2. 执行一部正向传播得出结果，$a^{<1>}$和$\hat y^{<1>}$，计算公式如下：

$$a^{<t+1>} = tanh(W_{ax}x^{<t+1>}+W_{aa}a^{<t>}+b) \tag{1}$$

$$ z^{<t + 1>} = W_{ya}  a^{<t + 1>} + b_y \tag{2}$$

$$ \hat{y}^{<t+1>} = softmax(z^{<t + 1>})\tag{3}$$

$ \hat{y_i}^{<t+1>}$表示了索引为i的那个字母是这个时间步输出结果的可能性。
3. 执行采样：根据$\hat{y}^{<t+1>}$计算出来的概率分布，挑选出一个字母索引。例如：$ \hat{y_i}^{<t+1>} = 0.16$，说明选择第i个索引的字母的可能性为16%。
`np.random.choice()`分布的作用就是在这样一个概率分布下进行采样。
4. 下一步就是覆盖变量x,将预测后的结构对应的字母使用one-hot法变成向量，机组正向传播，直到得到一个“\n”字符，表示单词也就是名字结束。

In [9]:
# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size,1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0      # counter 用来记录一个名字的最多字母数，不超过50个，newline_character是个标志，表示不和结束标志相等，则循环继续
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        # 对于已经有了关于x和a的输入，那么就开始计算a和y值
        a = np.tanh(np.dot(Wax,x)+ np.dot(Waa,a_prev)+b)
        z = np.dot(Wya,a)+by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(range(len(y)),p=y.ravel())
        # np.random.choice()方法表示在一个概率分布里面进行采样，首先range(len(y))表示在从0到长度这个多个数字中进行索引，后面的p参数表示这些数字对应的概率分部
        # 这个方法最后会返回一个采样结果的id号

        # Append the index to "indices"
        # 将采样结果索引好放入整个结果中，便于后面映射为字母
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        # 将下一个单元的输入变成上一个单元的输出，即one-hot形式，就相当于对应字母位置为1，其余为0
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        # 并且设置上一个单元a为上一步计算出来的，便于循环
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [10]:
np.random.seed(2)
n, n_a = 20, 100
a0 = np.random.randn(n_a, 1)
i0 = 1 # first character is ix_to_char[i0]
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])

Sampling:
list of sampled indices: [18, 2, 26, 0]
list of sampled characters: ['r', 'b', 'z', '\n']


## 3 建立语言模型

### 3.1 梯度下降

实现一个函数来执行随机梯度下降法，并且有裁剪梯度操作。一次只会训练一个仰恩，所以优化算法使用随机梯度下降法。下面是一个RNN通用优化循环步骤：
1. 在RNN上正向传播计算损失
2. 反向传播计算参数梯度
3. 修剪梯度，如果必要的话
4. 使用梯度下降法更新参数

We provide you with the following functions: 

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in the backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```


In [11]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ### START CODE HERE ###
    
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    return loss, gradients, a[len(X)-1]

In [12]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

Loss = 126.50397572165345
gradients["dWaa"][1][2] = 0.19470931534725341
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032004315
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]


### 3.2 训练模型

在给定的恐龙名字数据集中，我们用数据集的每一行，也就是一个名字作为一个样本。随机梯度下降的每100步，将抽取10个随机选择的名字来看算法是怎么做的。需要随机打乱数据集，以致于随机梯度下降法以随机次序访问样本。

当`examples[index]`包含一个恐龙名字是，创建一个样本（X，Y），可以使用下面代码：

```python
index = j % len(examples)
X = [None] + [char_to_ix[ch] for ch in examples[index]] 
Y = X[1:] + [char_to_ix["\n"]]

# 其中j分别取1，，，，num_iterations，来保证examples[index]是一个有效的，index是小于len(examples)的，%是取余。每一迭代只随机选择一个样本，而迭代次数可能是大于总的样本个数的，因此就需要取余操作。其中X是取出来的这个样本的每个字母所对应的数字，并且让X的第一个字符为空，接下来会被rnn_forward()函数中的0向量替换掉。而Y则根据对应的RNN模型法则，保证上一个单元的输出是下一个单元的输入，所以错位，并且最后一个字符是结束符。

```

In [13]:
# GRADED FUNCTION: model

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration.
    在每一次迭代的时候想要采样的恐龙名字的个数
    
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    # 根据对应的a的维度和x的维度，初始化5个参数
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    # strip()方法移除字符串头尾指定的字符，空格或者换行符
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        ### START CODE HERE ###
        
        # Use the hint above to define one training example (X,Y) (≈ 2 lines)
        index = j% len(examples)
        X = [None]+[char_to_ix[ch] for ch in examples[index]]
        Y = X[1:]+[char_to_ix['\n']]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01'
        curr_loss, gradients, a_prev= optimize(X,Y,a_prev,parameters,learning_rate=0.01)
        
        ### END CODE HERE ###
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        # 使用一个延迟技巧来保证loss函数平滑，可以加速训练过程。
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        # 后面这一部分表示，在训练过程中，每迭代2000词，就抽样中7个名字。
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                #  采样方法是根据参数，字母和索引对应表 通过初始化a和x为0，按照规则对每一个时间步进行采样，直到生成一个含50个字母或者提前遇到结束符截止的一系列字母的索引号列表
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
            print('\n')

In [14]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.084040

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


Iteration: 2000, Loss: 27.885977

Nivusajomoraurus
Ipda
Ivusccoloravhusacros
Nabalosagatris
Xusednesanvgortarchisiantixdganaluhamanfagoktonosa
Ca
Troipermaurusabropishantixemanaluicgandaloltonosan


Iteration: 4000, Loss: 25.934257

Ngytos
Kkedalosanchtesaurus
Lytosaurus
Ndaaisikacisaurus
Wusodonopgveros
Caahosanchtesaurus
Torangoraviros


Iteration: 6000, Loss: 24.801474

Onyuskanesbaunosebranisalrus
Lmbaagron
Lyutohiepleus
Olaahus
Xutodikus
Ecahsteechus
Trocheosaurus


Iteration: 8000, Loss: 24.186997

Onvtropherilus
Lngaanrtel
Lyuspenatopthurrasaurus
Omaalosaurus
Xutomigis
Egalosaurus
Trodonasaurus


Iteration: 10000, Loss: 23.663240

Miusuraperekups
Ingabertedan
Iusuraperekups
Mecalosaurus
Yutolonipeultos
Daahsteia
Trteratops


Iteration: 12000, Loss: 23.608030

Niwosaurus
Jikaadosaurus
Kystodon
Ngbaiptodan
Yusicheriathusacongisagrosthia
Dabasid

## **疑点**:
    根据原本的代码：optimize(X,Y,a_prev,parameters,learning_rate=0.01)的返回值没有包含parameters，从而觉得在整个训练迭代过程中，参数都没有更新，都是使用的初始化后的参数，即形参的问题。根据测试，发现，初始化后的参数，参数parameters变量的结果发生改变（不是特别确定，有点懵），欢迎其他学者一起讨论，批评指正。

- 接下来引出关于函数不返回参数，但是在传入数组，并且内部有对内存中的内容进行修改操作时，不返回也能修改其内容，但是对于数值型参数，只是简单的将参数名标签指向某个地址，如果不返回，内容没有修改。所以对于上面的关于恐龙命名中，训练过程是更改了参数的。

案例：
```python
def ddd(para):
    para[0]+=10
  
para = [5,6,7]
ddd(para)
ptint(para)
```

那么就有其他的问题：
- 1.那以前写过的那些深度学习训练的过程代码，是不是都不需要返回parameters参数的?
- 2.为什么要返回gradients 却不用返回parameters呢？

以上两个问题暂时都很懵，等以后知识储备多了说不定就懂了。

### 关于命名实体类问题的总结：

首先本次训练过程使用的是随机梯度下降法，所以每次计算梯度只针对于一个样本，所以对于训练数据和标签的制作是针对一个一个的名字，每一个名字就是一个样本，所以X是二维的，Y也是，并且按照1对1序列化模型，Y和X是错位的，并且增添了一个EOS结束符。

模型在训练过程中的迭代一次只对一个样本进行操作，关于RNN模型反向传播梯度计算方法，是通过最后一个RNN单元计算出来的代价，y是通过a求出来的，所以可以从代价到a，再到各个参数进行求导数，将所有的时间步的参数的值累加起来，就是这次迭代过程中反向传播计算出来的导数，通过该梯度值来完成参数的更新。


## 4 莎士比亚文风

生成莎士比亚文风的诗跟生成名字类似，但是更加复杂，数据集应该更换成莎士比亚的诗，使用LSTM单元，以便学到诗里面的更多依赖关系。

下面Keras代码已经实现了生成莎士比亚诗。

In [15]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

Loading text data...
Creating training set...
number of training examples: 31412
Vectorizing training set...
Loading model...


C:\python\anaconda3\lib\site-packages\keras\models.py:318: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


在上面引入包中，其中shakespeare_utils.py完成了加在数据文件，并且制作训练数据和对应的标签的工作，并且直接加载了训练好的模型，从'models/model_shakespeare_kiank_350_epoch.h5'文件中，

为了节约时间，已经训练好了一个模型，在the Sonnets诗集上迭代了1000次，接下来再训练这个模型1次，需要输入40个字母，开始的时候需要提供一个句子，接下来模型就会生成剩余的部分。

In [16]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

Epoch 1/1
31412/31412 [==============================] - 20s 642us/step - loss: 2.7131


In [17]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: Tianyan is a big pig


Here is your poem: 

Tianyan is a big pigtise and,
i will every deat beat oh the astorns benter,
and makn thy sore with yut dipard, and ase, and sider,
make the bonot bain mataunted a lime;
for hiw rost ourm live heunr sire mind,
not which that the cartoon of i spost's bistarred,
my so thy boind thas bodins, lire the conblesting sis,
a telles a ding that whee i selpore live,
that loed my hand the mascare weves to wrand
cinver that is whe

RNN-Shakespeare 模型与生成恐龙名字的模型很像，不同点在于：
1. LSTM替代了基本RNN模型，为了捕捉长范围的依赖关系
2. 模型更深，堆叠了两层
3. 使用了Keras模型

If you want to learn more, you can also check out the Keras Team's text generation implementation on GitHub: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py.

**References**:
- This exercise took inspiration from Andrej Karpathy's implementation: https://gist.github.com/karpathy/d4dee566867f8291f086. To learn more about text generation, also check out Karpathy's [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).
- For the Shakespearian poem generator, our implementation was based on the implementation of an LSTM text generator by the Keras team: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py 

#### 补充笔记

关于__future__模块中的函数：在老版本中兼顾新特性的一种方法，从python2.1开始以后, 当一个新的语言特性首次出现在发行版中时候, 如果该新特性与以前旧版本python不兼容, 则该特性将会被默认禁用. 如果想启用这个新特性, 则必须使用 "from __future__import *" 语句进行导入.

python set():作用是将文本字符串的所有出现的字母变成一个集合,集合还有add和update方法

关于莎士比亚作诗的训练数据和样本，训练数据是个三维的（样本个数，词库数，one-hot向量），标签是最后一个字符的one-hot向量，并且使用的是直接训练好的模型，所以直接加载模型即可。